# Equity Linked Notes - Autocallable Phoenix (basket)

This notebook demonstrates how to access and use the functionalities of **Autocallable Structured Notes** which are part of our **QPS** module within LSEG Financial Analytics SDK. This notebook presents a Phoenix structure with basket underlying.

**You will be able to:**
* Define Phoenix instrument with basic parameters
* Configure pricing parameters
* Evaluate Phoenix Analytics

## Imports

Import the following necessary modules:

- `lseg_analytics.pricing.instruments.structured_products` - for Structured Products instruments definitions and analytics

This notebook uses external libraries **pandas, IPython**; please ensure they are installed in your Python environment (e.g. 'pip install pandas') before running the code.

In [1]:
from lseg_analytics.pricing.instruments import structured_products as sp 

import json
import datetime as dt
import pandas as pd
from IPython.display import display

## Data Preparation

To define a Structured Product instrument, in this example Autocallable Phoenix, you need to follow a structured 3-step process:
* **Structured Product Definition** - Specify basic Structured Product parameters (strike, dates, notional, index)
* **Structured Product Instrument Definition** - Create the instrument object
* **Pricing Preferences** - Configure pricing parameters, optional

In [2]:
# 1. Create SP definition object

SN_definition = sp.StructuredProductsDefinition(
    deal_ccy = "EUR",
    instrument_tag = "Phoenix_basket",
    inputs = [
        sp.NameTypeValue(name="notional", type = "string", value="1000"),
        sp.NameTypeValue(name="Basket", type = "string", value="MBGn_DE|BMWG_DE"),
        sp.NameTypeValue(name="FirstAsset", type = "string", value="Perf_MBGn_DE"),
        sp.NameTypeValue(name="LastAsset", type = "string", value="Perf_BMWG_DE"),
        sp.NameTypeValue(name="BasketFunction", type = "string", value="Average"),
        sp.NameTypeValue(name="BasketPerf", type = "string", value="If(\"BasketFunction\" == \"WorstOf\", Min(FirstAsset[t]:LastAsset[t]), IF(\"BasketFunction\" == \"Average\", Average(FirstAsset[t]:LastAsset[t],True), 0))"),
        sp.NameTypeValue(name="StrikeDate", type = "date", value="26/08/2025"),
        sp.NameTypeValue(name="MaturityDate", type = "date", value="04/09/2034"),
        sp.NameTypeValue(name="LastValuationDate", type = "date", value="28/08/2034"),
        sp.NameTypeValue(name="Schedule", type = "schedule", value=[
                    ["26/02/2026", "26/02/2026", "05/03/2026", "05/03/2026"],
                    ["26/08/2026", "26/08/2026", "02/09/2026", "02/09/2026"],
                    ["26/02/2027", "26/02/2027", "05/03/2027", "05/03/2027"],
                    ["26/08/2027", "26/08/2027", "02/09/2027", "02/09/2027"],
                    ["28/02/2028", "28/02/2028", "06/03/2028", "06/03/2028"],
                    ["28/08/2028", "28/08/2028", "04/09/2028", "04/09/2028"],
                    ["26/02/2029", "26/02/2029", "05/03/2029", "05/03/2029"],
                    ["27/08/2029", "27/08/2029", "03/09/2029", "03/09/2029"],
                    ["26/02/2030", "26/02/2030", "05/03/2030", "05/03/2030"],
                    ["26/08/2030", "26/08/2030", "02/09/2030", "02/09/2030"],
                    ["26/02/2031", "26/02/2031", "05/03/2031", "05/03/2031"],
                    ["26/08/2031", "26/08/2031", "02/09/2031", "02/09/2031"],
                    ["26/02/2032", "26/02/2032", "04/03/2032", "04/03/2032"],
                    ["26/08/2032", "26/08/2032", "02/09/2032", "02/09/2032"],
                    ["28/02/2033", "28/02/2033", "07/03/2033", "07/03/2033"],
                    ["26/08/2033", "26/08/2033", "02/09/2033", "02/09/2033"],
                    ["27/02/2034", "27/02/2034", "06/03/2034", "06/03/2034"]]),
        sp.NameTypeValue(name="CouponBarrier", type = "curve", value=[["26/08/2025","70%"]]),
        sp.NameTypeValue(name="FinalCouponBarrier", type = "string", value="70%"),
        sp.NameTypeValue(name="AutocallBarrier", type = "curve", value=[
                    ["26/02/2027","100%"],
                    ["26/08/2027","97.5%"],
                    ["28/02/2028","95%"],
                    ["28/08/2028","92.5%"],
                    ["26/02/2029","90%"],
                    ["27/08/2029","87.5%"],
                    ["26/02/2030","85%"],
                    ["26/08/2030","82.5%"],
                    ["26/02/2031","80%"],
                    ["26/08/2031","80%"],
                    ["26/02/2032","80%"],
                    ["26/08/2032","80%"],
                    ["28/02/2033","80%"],
                    ["26/08/2033","80%"],
                    ["27/02/2034","80%"]]),
        sp.NameTypeValue(name="RedemptionBarrier", type = "string", value="50%"),
        sp.NameTypeValue(name="CouponRate", type = "string", value="4.50%"),
        sp.NameTypeValue(name="Leverage", type = "string", value="100%"),
        sp.NameTypeValue(name="VarInit", type = "string", value="0"),
        sp.NameTypeValue(name="NbPeriodNoCallable", type = "string", value="2")
    ],
    payoff_description = [
          [
            "Schedule type",
            "Schedule description",
            "Repeat(Basket,#)",
            "Repeat(Basket,Perf_#)",
            "Performance",
            "Alive",
            "Early",
            "Count",
            "Coupon",
            "Settlement",
            "OptionAtMaturity",
            "PriceIn%",
            "Price"
          ],
          [
            "AtDate",
            "StrikeDate",
            "EqSpot(#)",
            "",
            "",
            "1",
            "",
            "$n=VarInit;$n",
            "",
            "",
            "",
            "",
            ""
          ],
          [
            "OnUserSchedule",
            "Schedule",
            "EqSpot(#)",
            "#[t]/#[StrikeDate]",
            "BasketPerf",
            "If($n<NbPeriodNocallable,1,If(Performance[t]>=Interpol(AutocallBarrier,PS()),0,Alive[LastDate]))",
            "(1 - Alive[t]) * Alive[LastDate(-1)]",
            "$n=$n+1;$n",
            "Receive If(Performance[t]>=Interpol(CouponBarrier,PS()),CouponRate,0)*Alive[LastDate(-1)]",
            "Receive Early[t]",
            "",
            "",
            ""
          ],
          [
            "AtDate",
            "LastValuationDate",
            "EqSpot(#)",
            "#[t]/#[StrikeDate]",
            "BasketPerf",
            "",
            "",
            "$n=$n+1;$n",
            "Receive (MaturityDate, If(Performance[t]>=FinalCouponBarrier, CouponRate, 0) * Alive[LastDate])",
            "Receive (MaturityDate, Alive[LastDate])",
            "Receive (MaturityDate, If(Performance[t]>=RedemptionBarrier,0,(Performance[t] / Leverage -1)*Alive[LastDate]))",
            "Report((columnval(Coupon)+columnval(OptionAtMaturity)+columnval(Settlement))*100)",
            "Report(columnval(PriceIn%)/100*Notional)"
          ]
        ]
)


# 2. Create SP instrument definition object

phoenix_basket = sp.StructuredProductsDefinitionInstrument(definition = SN_definition)
print("Instrument definition created")


# 3. Create SP parameters object - optional

phoenix_basket_pricing_params = sp.StructuredProductsPricingParameters(
    valuation_date= dt.date(2025, 10, 15),  # Set your desired valuation date
    report_ccy="EUR",  # Set your reporting currency
    numerical_method = sp.GenericNumericalMethod(method="MonteCarlo"),
    models=[sp.ModelDefinition(
                  underlying_code = "MBGn.DE",
                  underlying_name = "MBGn_DE",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire"),
            sp.ModelDefinition(
                  underlying_code = "BMWG.DE",
                  underlying_name = "BMWG_DE",
                  underlying_currency = "EUR",
                  asset_class = "Equity",
                  model_name= "Dupire")
          ]
)

print("Pricing parameters configured")

Instrument definition created
Pricing parameters configured


## Request Execution

In [3]:
# Execute the calculation using the price() function
# The 'definitions' parameter accepts a list of instruments definitions for batch processing
try:
    response = sp.price(
        definitions=[phoenix_basket],
        pricing_preferences=phoenix_basket_pricing_params,
        market_data=None,
        return_market_data=True,  # or False
        fields=None  # or specify fields as a string
    )
    
    errors = [a.error for a in response.data.analytics if a.error]
    if errors:
        raise Exception(errors[0].message)
    print("Pricing execution completed")
    
except Exception as e:
    print(f"Price calculation failed: {str(e)}")
    raise

Pricing execution completed


## Results Display

#### Key Sections in the `response` JSON

 - **definitions**: Instrument setup (e.g., strike, dates, notional, underlying, barrier, profit target), it's StructuredProductDefinition that we used. 

 - **pricingPreferences**: Valuation date, financial model, numerical method.

 - **analytics**:
   - **tabularData**: `data`, `headers`, `statuses`
   - **cashflows**: Includes arrays and detailed `cashFlows` (payments, fixings)
   - **description**: Instrument summary StructuredProductDefinition and also the default fields not specified in the StructuredProductDefinition, but used by default in the calculation
   - **greeks**: Sensitivities like `deltaAmountInDealCcy`, `gammaAmountInDealCcy`, `thetaAmountInDealCcy`, `vegaAmountInDealCcy`
   - **pricingAnalysis**: `valuationDate`, `marketDataDate`
   - **valuation**: `marketValueInDealCcy`
   - **error**: Empty if no issues

#### Description
Useful for understanding which fields are included by default in the price function, even if they are not explicitly specified.

In [4]:
# Extract description from response
description = response.data.analytics[0].description

# Convert to dictionary for display
print(json.dumps(description.as_dict(), indent=4))

{
    "instrumentTag": "Phoenix_basket",
    "dealCcy": "EUR",
    "reportCcy": "EUR",
    "discountCurveId": "IRCurve_EUREURIBORSwapZCCurve_0001-01-01T00:00:00",
    "discountCurveName": "EUR EURIBOR Swap ZC Curve",
    "outputList": {
        "Alive": 8.440722433,
        "BMWG_DE": 1066.405242098,
        "Count": 149.244290364,
        "Coupon": 0.180812465,
        "Early": 0.575647607,
        "MBGn_DE": 686.42307621,
        "OptionAtMaturity": -0.208918356,
        "PERF_BMWG_DE": 11.786087998,
        "PERF_MBGn_DE": 12.706832214,
        "Performance": 12.246460106,
        "Price": 866.737948871,
        "PriceIn%": 86.673794887,
        "Settlement": 0.894843839
    },
    "processingInformation": "'PricePercent' column was not retrieved in payoff description: related fields will not be retrieved. The following greeks could not be retrieved: Theta, Vega, Gamma, Delta. Make sure a 'PricePercent' column is specified in payoff description to compute greeks. "
}


#### Analytics

##### Valuation

In [5]:
# Extract vauation from the response
valuation = response.data.analytics[0].valuation

# Convert the dictionary to a DataFrame
df_phoenix_basket_valuation = pd.DataFrame(list(valuation.items()), columns=["Field", "Value"])

display(df_phoenix_basket_valuation)

,Field,Value
0,marketValueInDealCcy,866.737949
1,marketValueInReportCcy,866.737949


##### Cash Flows

In [6]:
# Extract cashflows from response
cashflows = response.data.analytics[0].cashflows["cashFlows"]

# Extract underlyings
model_df = pd.DataFrame(data=phoenix_basket_pricing_params.models)
underlying_list = model_df['underlyingName'].to_list()

# Build dataframes for all cash flow types
output = []
for cf_type in cashflows:
    cashflow_df = pd.DataFrame(cf_type['payments'])
    if cf_type['legTag'] == 'Index':
        cashflow_df = cashflow_df.rename(columns={'amount': underlying_list[0]})
    else:
        cashflow_df = cashflow_df.rename(columns={'amount': cf_type['legTag']})
    cashflow_df['discountFactor'] = cashflow_df['discountFactor'].round(4)
    output.append(cashflow_df)

# Merge all dataframes on the 'date' column
combined_df = pd.concat([*output],axis=1)

# Remove duplicated columns with the same values
combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]

common_cols = ['date', 'discountFactor', 'Performance', 'Alive', 'Early', 'Coupon', 'Settlement', 'OptionAtMaturity', 'Price', 'currency', 'occurence']
indv_perf = [f'PERF_{underlying}' for underlying in underlying_list if len(underlying_list) > 1]
cols_to_display = common_cols[0:2] + underlying_list + indv_perf + common_cols[2:]

# Leave only columns to display
combined_df = combined_df.loc[:,[*cols_to_display]]

# Display the combined dataframe
display(combined_df)

,date,discountFactor,MBGn_DE,BMWG_DE,PERF_MBGn_DE,PERF_BMWG_DE,Performance,Alive,Early,Coupon,Settlement,OptionAtMaturity,Price,currency,occurence
0,2025-08-26,1.0000,54.020000,90.480000,0.000000,0.000000,0.000000,1.0000,0.0000,0.000000,0.000000,0.000000,0.000000,EUR,Historical
1,2026-02-26,0.9926,52.711553,79.313192,0.975778,0.876583,0.926181,1.0000,0.0000,0.043552,0.000000,0.000000,0.000000,EUR,Future
2,2026-08-26,0.9832,50.724385,76.170022,0.938993,0.841844,0.890418,1.0000,0.0000,0.038722,0.000000,0.000000,0.000000,EUR,Future
3,2027-02-26,0.9740,51.211201,76.803777,0.948004,0.848848,0.898426,0.6902,0.3098,0.035987,0.309688,0.000000,0.000000,EUR,Future
4,2027-08-26,0.9647,48.678111,73.278162,0.901113,0.809882,0.855498,0.6256,0.0646,0.018992,0.064576,0.000000,0.000000,EUR,Future
5,2028-02-28,0.9548,49.185833,73.996265,0.910512,0.817819,0.864165,0.5576,0.0680,0.015438,0.067973,0.000000,0.000000,EUR,Future
6,2028-08-28,0.9448,46.382622,69.780437,0.858619,0.771225,0.814922,0.5216,0.0360,0.009086,0.035985,0.000000,0.000000,EUR,Future
7,2029-02-26,0.9344,46.701427,70.897637,0.864521,0.783572,0.824047,0.4854,0.0362,0.007656,0.036184,0.000000,0.000000,EUR,Future
8,2029-08-27,0.9237,43.247857,66.176686,0.800590,0.731396,0.765993,0.4712,0.0142,0.004021,0.014194,0.000000,0.000000,EUR,Future
9,2030-02-26,0.9127,43.566862,66.878781,0.806495,0.739155,0.772825,0.4556,0.0156,0.003832,0.015593,0.000000,0.000000,EUR,Future


##### Greeks

In [7]:
# Extract Greeks from the response
greeks = response.data.analytics[0].greeks

# Convert the dictionary to a DataFrame
df_phoenix_basket_greeks = pd.DataFrame(list(greeks.items()), columns=["Greeks", "Value"])

display(df_phoenix_basket_greeks)

,Greeks,Value
0,deltaAmountInDealCcy,11.848800
1,deltaAmountInReportCcy,11.848800
2,gammaAmountInDealCcy,0.827406
3,gammaAmountInReportCcy,0.827406
4,vegaAmountInDealCcy,-6.387820
5,vegaAmountInReportCcy,-6.387820
6,thetaAmountInDealCcy,0.490605
7,thetaAmountInReportCcy,0.490605
